참고  
* https://www.maartengrootendorst.com/blog/bertopic/  
* https://hackernoon.com/nlp-tutorial-topic-modeling-in-python-with-bertopic-372w35l9

In [1]:
!pip install bertopic[visualization]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

SBERT를 이용한 토픽 모델인 BERTopic은 별도 논문은 나오지 않은 모델이지만, 개발자는 BERTopic이 LDA를 대체할 수 있을만큼의 기술이라고 확신한다고 합니다. 여기서는 BERT 기반의 토픽 모델링 구현체인 BERTopic의 간단한 사용 방법에 대해서 다룹니다.

# 1. BERTopic

BERTopic은 BERT embeddings과 클래스 기반(class-based) TF-IDF를 활용하여 주제 설명에서 중요한 단어를 유지하면서도 쉽게 해석할 수 있는 조밀한 클러스터를 만드는 토픽 모델링 기술입니다. BERTopic 알고리즘은 크게 세 가지 과정을 거칩니다.

**1) 텍스트 데이터를 SBERT로 임베딩합니다.**  
알고리즘은 SBERT를 사용하여 문서를 임베딩합니다. 이때, BERTopic은 기본적으로 아래의 BERT들을 사용합니다.  

* **"paraphrase-MiniLM-L6-v2"** : 영어 데이터로 학습된 SBERT
* **"paraphrase-multilingual-MiniLM-L12-v2"** : 50개 이상의 언어로 학습된 다국어 SBERT

**2) 문서를 군집화합니다.**  
UMAP을 사용하여 임베딩의 차원을 줄이고 HDBSCAN 기술을 사용하여 차원 축소된 임베딩을 클러스터링하고 의미적으로 유사한 문서 클러스터를 생성합니다.

**3) 토픽 표현을 생성**  
마지막 단계는 클래스 기반 TF-IDF로 토픽을 추출합니다.

# 2. 데이터 로드

In [2]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

In [3]:
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

In [4]:
# 상위 5개의 샘플 출력
docs[:5]

["\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n",
 'My brother is in the market for a high-performance video card that supports\nVESA local bus with 1-2MB RAM.  Does anyone have suggestions/ideas on:\n\n  - Diamond Stealth Pro Local Bus\n\n  - Orchid Farenheit 1280\n\n  - ATI Graphics Ultra Pro\n\n  - Any other high-per

In [5]:
print('총 문서의 수 :', len(docs))

총 문서의 수 : 18846


# 3. 토픽 모델링

BERTopic의 fit_transform 메소드에 문자열들의 리스트를 입력으로 넣으면 토픽 모델링을 수행합니다.

In [6]:
model = BERTopic()
topics, probabilities = model.fit_transform(docs)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
print('각 문서의 토픽 번호 리스트 :',len(topics))
print('첫번째 문서의 토픽 번호 :', topics[0])

각 문서의 토픽 번호 리스트 : 18846
첫번째 문서의 토픽 번호 : 0


In [8]:
# 각 토픽의 크기와 각 토픽에 할당된 단어들
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6538,-1_to_the_and_of,"[to, the, and, of, is, for, you, it, in, that]",[[to Benedikt Roseneau ]\n\n#In article <1qv6a...
1,0,1841,0_game_team_games_he,"[game, team, games, he, players, season, hocke...",[1992-93 Los Angeles Kings notes and game repo...
2,1,614,1_key_clipper_chip_encryption,"[key, clipper, chip, encryption, keys, governm...",[The following document summarizes the Clipper...
3,2,531,2_ites_hello_cheek_hi,"[ites, hello, cheek, hi, yep, huh, ken, art, w...","[Hi,, \n \n ..."
4,3,342,3_israel_israeli_arab_arabs,"[israel, israeli, arab, arabs, jews, palestini...","[\n\n""Assuming""? Also: come on, Brad. If we ar..."
...,...,...,...,...,...
209,208,11,208_chemistry_paperback_book_guide,"[chemistry, paperback, book, guide, petrucci, ...",[SOMEONE PLEASE BUY THESE BOOKS!!!!! I AM NOT...
210,209,11,209_image_processing_analysis_tools,"[image, processing, analysis, tools, plplot, d...",[Archive-name: x-faq/part4\nLast-modified: 199...
211,210,11,210_meat_carcinogenic_smoke_carcinogens,"[meat, carcinogenic, smoke, carcinogens, charc...",[\nSome recent postings remind me that I had r...
212,211,10,211_fonts_pi_fences_subscripts,"[fonts, pi, fences, subscripts, select, iig, g...","[\nIT'S BACK!!!!!!\n\nCongratulations, you've ..."


In [9]:
model.get_topic_info()['Count'].sum()

np.int64(18846)

위의 출력에서 Topic -1이 가장 큰 것으로 보입니다. -1은 토픽이 할당되지 않은 모든 이상치 문서(outliers)들을 나타냅니다. 현재 0번 토픽부터 210번 토픽까지 있는데, 임의로 5번 토픽에 대해서 단어들을 출력해봅시다.  

get_topic() 메소드의 입력으로 보고자하는 토픽의 번호를 넣어줍니다.

In [10]:
model.get_topic(5)

[('bike', np.float64(0.01452824254099266)),
 ('riding', np.float64(0.011373073317084915)),
 ('ride', np.float64(0.010066682902822333)),
 ('lane', np.float64(0.008527170989923977)),
 ('my', np.float64(0.00838210431387999)),
 ('passenger', np.float64(0.007410650713360553)),
 ('you', np.float64(0.007250381394324716)),
 ('road', np.float64(0.007038849710654006)),
 ('car', np.float64(0.006933338610348845)),
 ('traffic', np.float64(0.006527001508633047))]

# 4. 토픽 시각화

BerTopic을 사용하면 LDAvis와 매우 유사한 방식으로 생성된 토픽을 시각화할 수 있습니다. 시각화를 통해 생성된 토픽에 대해 더 많은 통찰력을 얻을 수 있습니다. 우선 visualize_topics() 메소드로 시각화를 진행해봅시다.

In [11]:
model.visualize_topics()

# 5. 단어 시각화

Visualization_barchart() 메소드는 c-TF-IDF 점수에서 막대 차트를 만들어 각 토픽에 대해 선택된 단어들을 표시합니다. 각 토픽에 대해서 선택된 단어들을 비교할 수 있습니다.

In [12]:
model.visualize_barchart()

# 6. 토픽 유사도 시각화

각 토픽들이 서로 얼마나 유사한지 시각화할 수도 있습니다. visualize_heatmap() 메소드를 사용하여 히트맵을 시각화합니다.  
해당 히트맵의 원하는 위치에 마우스를 갖다대면 실질적인 유사도 값을 확인할 수 있습니다.

In [13]:
model.visualize_heatmap()

# 7. 토픽의 수 정하기

때때로 너무 많은 토픽이 생성되거나 너무 적은 토픽이 생성될 수 있습니다. 토픽의 수를 직접 정하고 싶다면 몇 가지 방법이 존재합니다.

첫번째 방법은 모델 객체 생성 시에 nr_topics 값으로 원하는 토픽 수를 입력하여 원하는 토픽의 수를 설정할 수 있습니다. BerTopic은 유사한 토픽들을 찾아 하나의 토픽으로 병합합니다.

In [14]:
model = BERTopic(nr_topics=20)
topics, probabilities = model.fit_transform(docs)

In [15]:
model.visualize_topics()

또 다른 방법은 모델이 자동으로 토픽의 수를 줄이도록 설정하는 것입니다. 이 옵션을 사용하려면 모델 객체 생성 시에 "nr_topics"의 값을 "auto"로 설정하면 됩니다.

In [16]:
model = BERTopic(nr_topics="auto")
topics, probabilities = model.fit_transform(docs)

In [17]:
# 각 토픽의 크기와 각 토픽에 할당된 단어들
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6579,-1_the_to_of_and,"[the, to, of, and, is, in, that, it, for, you]","[First, I thank collectively all people who ha..."
1,0,3291,0_for_is_and_the,"[for, is, and, the, with, to, it, or, you, on]",[Archive-name: x-faq/part4\nLast-modified: 199...
2,1,1838,1_game_team_he_games,"[game, team, he, games, the, was, players, sea...",[The FLYERS team that can beat any team on any...
3,2,834,2_were_of_the_they,"[were, of, the, they, and, in, was, armenian, ...","[\n\nIt is 'Serdar', 'kocaoglan'.\n\n\nJust lo..."
4,3,593,3_god_that_is_not,"[god, that, is, not, of, to, you, are, as, it]",[Note: I am breaking this reply into 2 parts ...
...,...,...,...,...,...
102,101,12,101_needles_needle_acupuncture_syringe,"[needles, needle, acupuncture, syringe, hypode...","[\n\nWhile I don't have an answer for you, I r..."
103,102,12,102_moscow_aviation_russian_kaliningrad,"[moscow, aviation, russian, kaliningrad, insti...","[\nCorrection, and some more info: The Kalinin..."
104,103,11,103_lines_airplanes_pistrix_balls,"[lines, airplanes, pistrix, balls, poles, elec...",[I got a question from my dad which I really c...
105,104,11,104_cancer_burzynskis_a10_cell,"[cancer, burzynskis, a10, cell, moss, informat...",[\nYou've asked a toughie of a question. Ther...


토픽의 개수를 지정하지 않았을 때는 0번 토픽부터 210번 토픽까지 총 211개의 토픽이 존재하였으나, 자동으로 토픽의 수가 줄어들도록 설정하자 토픽의 수가 0번부터 143번까지 총 144개로 줄어든 것을 확인할 수 있습니다.

# 8. 임의의 문서에 대한 예측

In [18]:
new_doc = docs[0]
print(new_doc)



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [19]:
topics, probs = model.transform([new_doc])

In [20]:
print('예측한 토픽 번호 :', topics)

예측한 토픽 번호 : [np.int64(1)]


In [21]:
model.save("20250807_강의복습_BERTopic_En")
BerTopic_model = BERTopic.load("20250807_강의복습_BERTopic_En")

2025-08-07 02:08:08,780 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
